In [1]:
import numpy as np
import re
import time
from stanfordcorenlp import StanfordCoreNLP
import torch
from torch.autograd import Function
from torch.autograd import Variable
import torch.nn as nn
from tqdm import tqdm
import time
import math
import logging
from sklearn.decomposition import PCA
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from sklearn import preprocessing
from random import shuffle
BATCH_SIZE = 512
EPOCHS = 401
LABELNUMS = 36

In [2]:
def make_one_hot(data1):
    return (np.arange(LABELNUMS)==data1[:,None]).astype(np.integer)

In [3]:
train_X = torch.from_numpy(np.load("train_Sentense_np.npy").astype("float32"))
train_Y = np.load("train_Relation_np.npy").astype("int32")
dev_X = torch.from_numpy(np.load("dev_Sentense_np.npy").astype("float32"))
dev_Y = np.load("dev_Relation_np.npy").astype("int32")

In [4]:
train_Y = train_Y.tolist()
dev_Y = dev_Y.tolist()

In [5]:
# OneHot
train_ontHot_y = []
dev_ontHot_y = []
for trainY in train_Y:
    tmp = np.eye(LABELNUMS)[trainY]
    tmp = tmp.sum(axis = 0).astype("int32")
    tmp = tmp[:-1]
    train_ontHot_y.append(tmp)
train_ontHot_y = torch.Tensor(train_ontHot_y)
print(train_ontHot_y.shape)
for devY in dev_Y:
    tmp = np.eye(LABELNUMS)[devY]
    tmp = tmp.sum(axis = 0).astype("int32")
    tmp = tmp[:-1]
    dev_ontHot_y.append(tmp)
dev_ontHot_y = torch.Tensor(dev_ontHot_y)
print(dev_ontHot_y.shape)

torch.Size([43501, 35])
torch.Size([38417, 35])


In [1]:
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F
class TEXTCNN(nn.Module):
    def __init__(self):
        super(TEXTCNN, self).__init__()
        self.conv1 = nn.Sequential(  # input shape (1, BATCH_SIZE, 200)
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=(1,200),
            ), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(50,1)), 
        )
        
        self.conv2 = nn.Sequential( 
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=(3,200), 
            ), 
            nn.ReLU(), 
            nn.MaxPool2d(kernel_size=(48,1)), 
        )
        
        self.conv3 = nn.Sequential( 
            nn.Conv2d(
                in_channels=1, 
                out_channels=16, 
                kernel_size=(5,200), 
                stride = 1,
            ), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(46,1)),
        )
        
        self.output = nn.Linear(48, LABELNUMS-1) 

    def forward(self, x):
        x = x.reshape((-1,1,x.shape[1],x.shape[2]))
        x1 = self.conv1(x)
        x2 = self.conv2(x)
        x3 = self.conv3(x)
        x = torch.cat((x1,x2,x3),1)
        x = x.view(x.size(0), -1)   # 展平
#         x = self.out(x)
        output = self.output(x)
        return output

In [7]:
model = TEXTCNN()
if torch.cuda.is_available():
    print("To GPU")
    train_X = train_X.cuda()
    train_ontHot_y = train_ontHot_y.cuda()
    dev_X = dev_X.cuda()
    dev_ontHot_y = dev_ontHot_y.cuda()
    model = model.cuda()
else:
    print("Still CPU")

To GPU


In [8]:
train = TensorDataset(train_X, train_ontHot_y)
dev = TensorDataset(dev_X, dev_ontHot_y)

In [13]:
train_loader = DataLoader(
    dataset = train, 
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 0, 
)


dev_loader = DataLoader(
    dataset = dev, 
    batch_size = BATCH_SIZE,
    shuffle = True, 
    num_workers = 0, 
)

In [19]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
loss_F = torch.nn.BCELoss()
sigmoid = nn.Sigmoid()
for epoch in range(1,EPOCHS): 
    start = time.time()
    losses = []
    #print("1")
    for step,(x,y) in enumerate(train_loader):
        predict = model(x)
        optimizer.zero_grad()
        loss = loss_F(sigmoid(predict), y)
        loss.backward()
        optimizer.step()
        losses.append(loss.data.mean())
    #print("2")
    with torch.no_grad():
        accs = []
        for step,(devx,devy) in enumerate(dev_loader):
            dev_pred = model(devx)
            dev_pred = sigmoid(dev_pred)
            tmp = dev_pred > 0.5
            count = 0
            for i in range(tmp.shape[0]):
                if torch.sum(tmp.int()[i] == devy.int()[i]).tolist() == devy.shape[1]:
                    count+=1
            acc = count / devy.shape[0]
            accs.append(acc)
        print(f"{epoch}: accuracy:{sum(accs)/len(accs)} loss: {torch.mean(torch.stack(losses))} time: {time.time() - start}")
    #print("flag")
    #assert False

1: accuracy:0.49992743808049533 loss: 0.055103663355112076 time: 4.840629816055298
2: accuracy:0.4867922189434985 loss: 0.053623706102371216 time: 4.857752561569214
3: accuracy:0.46971900396671823 loss: 0.05323966592550278 time: 4.895254850387573
4: accuracy:0.4269271236455109 loss: 0.05292476713657379 time: 4.868378400802612
5: accuracy:0.5339982827012384 loss: 0.052359458059072495 time: 4.868848562240601
6: accuracy:0.3503954624613003 loss: 0.05188378691673279 time: 4.861184358596802
7: accuracy:0.5069311750193498 loss: 0.051628097891807556 time: 4.855306386947632
8: accuracy:0.457820360874613 loss: 0.05127422884106636 time: 4.89635157585144
9: accuracy:0.4732201165828173 loss: 0.05089668557047844 time: 4.894102573394775
10: accuracy:0.5353119557856038 loss: 0.05054175481200218 time: 4.819168567657471
11: accuracy:0.4878232028831269 loss: 0.050170574337244034 time: 4.832513332366943
12: accuracy:0.5304518188854489 loss: 0.04991067945957184 time: 4.882385730743408
13: accuracy:0.46795

KeyboardInterrupt: 